In [1]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
import getpass
import os

In [2]:
cards_loader = CSVLoader("./data/cards.csv")
doc1_loader = PyPDFLoader("./data/doc1.pdf")
doc2_loader = PyPDFLoader("./data/doc2.pdf")

In [3]:
cards = cards_loader.load()
doc1 = doc1_loader.load()
doc2 = doc2_loader.load()

In [4]:
csv_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=0, add_start_index=True
)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50, add_start_index=True
)

In [5]:
card_splits = csv_splitter.split_documents(cards)
doc1_splits = text_splitter.split_documents(doc1)
doc2_splits = text_splitter.split_documents(doc2)

In [6]:
embeddings = OllamaEmbeddings(model="bge-m3")
vector_store = Chroma(embedding_function=embeddings)

In [7]:
_ = vector_store.add_documents(documents=card_splits)
print("Cards done")
_ = vector_store.add_documents(documents=doc1_splits)
print("doc1 done")
_ = vector_store.add_documents(documents=doc2_splits)
print("doc2 done")

Cards done
doc1 done
doc2 done


In [17]:
prompt = PromptTemplate.from_template(
    """
    Ты помощник-ассистент, который отвечает на вопросы клиента.
    Используй следующий контекст для информации и ответь на вопрос клиента.
    Если ты не знаешь или не нашел ответа, так и скажи.
    Если ты нашел больше одной информации, то подробно скажи о каждой из них.
    Вопрос: {question} 
    Контекст: {context} 
    Ответ:
    """
)

In [18]:
llm = ChatOllama(model="llama3.1:8b")

In [19]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    for doc in retrieved_docs:
        print(f"{doc.metadata}\n")
        print(f"{doc.page_content}\n")
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [22]:
result = graph.invoke({"question": "Что такое мобильный банк"})

{'page': 2, 'page_label': '3', 'source': './data/doc1.pdf', 'start_index': 3997}

использования Мобильного приложения Банка определяются настоящими Условиями и/или соглашением, с которым 
Клиент знакомится перед его использованием.
Мобильный Банк — сервис Банка, позволяющий осуществлять взаимодействие Банка и/или его партнеров 
и Клиента, включая обмен информацией и совершение отдельных операций, через специальные приложения для 
мобильного устройства (смартфона, планшета и т.п.).

{'page': 2, 'page_label': '3', 'source': './data/doc1.pdf', 'start_index': 3566}

для совершения Клиентом операций за счет предоставленного Банком Кредита без использования банковского 
счета Клиента.
Мобильное приложение Банка  — приложение для мобильного устройства (смартфона, планшета и т.п.), в том 
числе Мобильный Банк, позволяющее осуществлять взаимодействие Банка и/или его партнеров и Клиента, включая 
обмен информацией, направление Заявок и совершение отдельных операций по усмотрению Банка. Условия



In [23]:
print(f'Answer: {result["answer"]}')

Answer: Мобильный банк - это сервис банка, который позволяет клиентам выполнять различные операции через специальные приложения для мобильных устройств (смартфонов, планшетов и т.п.). В частности, с помощью Мобильного Банка можно обмениваться информацией, направлять заявки и совершать отдельные операции по усмотрению банка.

Мобильное приложение банка - это приложение для мобильных устройств, которое позволяет клиентам взаимодействовать с банком и его партнерами, включая обмен информацией, направление заявок и совершение отдельных операций.
